```
* 주의 : 현재 최신 버전의 tensorflow 를 돌리기 위해서는 python 3.5 필요. 3.6에서는 안돌아감. (최신 기준: 11월 20일. tensorflow version: tensorflow-1.4.0-cp35-cp35m-macosx_10_11_x86_64.whl)
```

# CNN (Convolution Neural Network, 합성곱 신경망)

- 대부분 이미지 처리에 활용
- "텐서플로 첫걸음" 5장 참고


## MNIST 손글씨 이미지 인식 문제

- MNIST 손글씨 이미지 예시: 
![title](images/mnist_plot.png)
    
- 각 data는 28 X 28 픽셀로 구성



### 00. MNIST 데이터 로드

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### 01. 텐서플로 플레이스 홀더 정의

- 텐서플로 플레이스 홀더 (tensorflow placeholder): 심볼릭 변수

In [3]:
import tensorflow as tf

x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

In [6]:
print (x)
print (y_)

Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)




### 02. 입력 데이터 재 정의

* 4D tensor로 재 정의. ? X Width X Height X Color channel (여기서는 흑백이라 -1)

In [10]:
x_image = tf.reshape(x, [-1, 28, 28, 1])
print ("x_image=")
print (x_image)

x_image=
Tensor("Reshape_1:0", shape=(?, 28, 28, 1), dtype=float32)


### 합성곱 신경망

![title](images/cnn_overview.png)
    
- 주요 목적: 테두리, 선, 색 등 이미지의 시각적 특징(characteristic)이나 특성(feature)을 감지
- 입력 계층 (input layer)과 은닉 계층(hidden layer)에 의해 처리됨

- 입력 뉴런의 작은 일부 영역만 첫 은닉 계층의 한 뉴런과 연결. (NOT FULLY connected)

- 예를 들어 5x5  영역이 한 뉴런과 연결된다면,
    - 28x28 을 훑기 위해서는 5x5 윈도우가 총 24x24개 필요

- Stride (스트라이드): 한 번에 얼만큼의 윈도우를 움직일 지 결정하는 매개변수
- Padding (패딩) : 이미지 바깥까지 윈도우가 이동하도록 허용할 때, 채울 테두리 크기 (넘어가는 값은 0 혹은 임의의 값.) 

- 합성곱 신경망에서는 y=Wx+b 에서, 은닉 계층의 모든 뉴런이 W와 b를 공유함
- kernel (filter, 커널 / 필터) : W, b
    - 고유한 특징을 찾는데 사용.
    - 하나의 커널은 이미지에서 한 종류의 특징만을 감지 (/ 혹은 | 등)
        - 커널로 특징맵 (feature map)을 만든다
    
- 합성곱 계층 (convolution layer): 특징맵 집합

- 풀링 계층 (pooling layer): 합성곱 계층에 따라오는 것
    - 합성곱 계층의 출력 값을 단순하게 압축하고, 합성곱 계층에서 생산한 정보를 컴팩트한 버전으로 만들어줌
    - 28x28 인풋을 5x5 윈도우로 훑으면 나오는 24x24 합성곱 결과를 2x2 영역으로 분할(max pooling) 하면 12x12 
    - 아래 코드에서는 padding 때문에 28x28 -> 28x28 (by padding) -> 14x14

In [12]:
# W
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# b
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [14]:
# convolution layer : stride(슬라이딩 윈도우 수) = 1, padding = SAME(패딩 방식중 하나)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# pooling 방식은 max pooling: 2x2 영역에서 가장 큰 값 선택
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [16]:
W_conv1 = weight_variable([5, 5, 1, 32]) # window가 5x5, 32개 필터(feature)
b_conv1 = bias_variable([32])

In [17]:
# 활성화 함수로 ReLU (Rectified Linear Unit) 사용: max(0, x) 리턴
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [18]:
print (h_conv1)

Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)


In [19]:
print (h_pool1)

Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


In [21]:
# 두번째 합성곱 계층. 예시를 위해 5x5 윈도우에 64개 필터로
# 이전 계층의 출력값을 받아야 해서 세번째 값이 32
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [22]:
print (h_conv2)

Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)


In [24]:
# 14x14 였던 h_pool1 에서 stride 1 적용 후 pooling 했기 때문에 7x7
print (h_pool2)

Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


In [26]:
# 완전 연결 계층에 연결하기 위해 1024개의 뉴런 사용

W_fc1 = weight_variable([7 * 7 * 64, 1024]) #h_pool2의 7x7 크기의 64개 필터, 임의로 설정한 1024개의 뉴런
b_fc1 = bias_variable([1024])

In [27]:
# tensor to vector
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [28]:
print (h_fc1)

Tensor("Relu_2:0", shape=(?, 1024), dtype=float32)


In [30]:
# dropout : 매개변수 줄이기 (overfitting 방지)

keep_prob = tf.placeholder("float") #뉴런이 dropout 되지 않을 확률 저장
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) # dropout 적용

In [31]:
print (keep_prob)

Tensor("Placeholder_3:0", dtype=float32)


In [32]:
print (h_fc1_drop)

Tensor("dropout_1/mul:0", shape=(?, 1024), dtype=float32)


### 마지막으로, 모델에 소프트맥스 계층 추가

* 여러 클래스로 데이터를 분류하고 싶을 때, sigmoid 함수의 일반화된 형태인 softmax 함수를 사용함

* 소프트맥스: 
$$evidence_i=\sum_{j} W_{i,j}x_j + b_i$$

$$y = softmax(evidence)$$

In [33]:
# 뉴런 수 1024, 숫자 0~9 클래스 10개
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [34]:
print (y_conv)

Tensor("Softmax:0", shape=(?, 10), dtype=float32)


In [35]:
# 훈련 및 평가
# 이전까지 우리는 아무것도 돌리지 않았습니다. '설정' 만 함.

In [36]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [39]:
# 우리는 여전히 아무것도 돌리지 않음. '선언'의 향연

In [41]:
sess = tf.Session() # 텐서플로우의 시작을 알림
sess.run(tf.initialize_all_variables()) # 드디어 돌림
for i in range(1000):
    batch = mnist.train.next_batch(100)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
        print ("step %d : training accuracy %g" % (i, train_accuracy))
    sess.run(train_step, feed_dict={x: batch[0], y_:batch[1], keep_prob:0.5})

print ("test accuracy: %g" % sess.run(accuracy, feed_dict={x: mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0 : training accuracy 0.1
step 100 : training accuracy 0.87
step 200 : training accuracy 0.89
step 300 : training accuracy 0.96
step 400 : training accuracy 0.93
step 500 : training accuracy 0.97
step 600 : training accuracy 0.95
step 700 : training accuracy 0.96
step 800 : training accuracy 0.93
step 900 : training accuracy 0.96
test accuracy: 0.9717


### 결과

(1000번까지 돌리면 날라갈것 같아서...)

![title](images/conv_result.png)


### 참고

http://cs231n.github.io/convolutional-networks/
https://www.tensorflow.org/versions/master/api_docs/python/tf/nn/conv2d
